In [356]:
data = pd.read_pickle('training_data.obj')
test = pd.read_csv('test.csv')

In [357]:
data['antiguedad'] = data['FechaAltaCliente'] - data['Año-Mes']
data.drop(columns=['FechaAltaCliente'],inplace=True)

In [386]:
cols = ['Hl_sum_3M','Dcto_porc_3M','Hl_sum_6M','Dcto_porc_6M','Hl_sum_1M','Dcto_porc_1M','Region','SubCanal','TipoPoblacion','Estrato','EF','antiguedad','Mes','Hl_sum_prodSim_3M', 'Dcto_porc_prodSim_3M', 'Hl_sum_prodSim_6M', 'Dcto_porc_prodSim_6M', 'Hl_sum_prodSim_1M', 'Dcto_porc_prodSim_1M','avg_6M_exec']
# Borrados: Duration, nProm
# Borrados: 
cats = ['Region','SubCanal','TipoPoblacion','Estrato','EF']
cats_bools = [False,False,False,False,False,False,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False]

training_data = data[data['Año-Mes'] > 0]
present_data = data[data['Año-Mes'] < 1]

In [387]:
training_data.sort_values(by='Año-Mes',ascending=False,axis=0)
training_data = training_data.reset_index(drop=True)
nrows = len(training_data.index)
breakpoint = (2*nrows)//3
data_train = training_data.iloc[0:breakpoint]
data_test = training_data.iloc[breakpoint:nrows]

x_test = data_test[cols]
y_test = data_test['Exec']
x_train = data_train[cols]
y_train = data_train['Exec']

In [380]:
eval_df = pd.merge(test,present_data,how='inner',on=['Marca','Cupo','Cliente'])
eval_df = eval_df[~eval_df.duplicated(['Marca','Cupo','Cliente'])]
x_eval = eval_df[cols]

In [361]:
# example of random oversampling to balance the class distribution
from collections import Counter
from imblearn.over_sampling import SMOTENC

sm = SMOTENC(categorical_features=cats_bools,random_state=12, sampling_strategy  = 1/6)
mat_train_res, y_train_res = sm.fit_sample(x_train,y_train)

x_train_res = pd.DataFrame(mat_train_res,columns=cols)

for col in cats:
    x_train_res[col] = x_train_res[col].astype('category')

In [ ]:
param_grid = {'max_bin':[255,400,500],
              'colsample_bytree':[0.64,0.65,0.66],
              'num_leaves':[8,16,24],
              'learning_rate':[0.005,0.01,0.03,0.05],
              'feature_fraction':[],
              'bagging_fraction':[],
              'reg_alpha':[1,1.2,1.4],
              'reg_lambda':[1,1.2,1.4],
              'subsample':[0.65,0.75,0.85]
              }

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_bin': 300,
    'colsample_bytree': 0.64,
    'num_leaves': 24,
    'learning_rate': 0.01,
    'reg_alpha': 1,
    'reg_lambda': 1.2,
    'subsample': 0.7}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                categorical_feature=cats)

preds = gbm.predict(x_eval)
test['Ejecuto_Promo'] = preds
test.to_csv('output_lgbm.csv',index=False)

In [389]:
list(zip(x_train.columns,gbm.feature_importance()))

[('Hl_sum_3M', 215),
 ('Dcto_porc_3M', 154),
 ('Hl_sum_6M', 416),
 ('Dcto_porc_6M', 308),
 ('Hl_sum_1M', 340),
 ('Dcto_porc_1M', 183),
 ('Region', 366),
 ('SubCanal', 145),
 ('TipoPoblacion', 21),
 ('Estrato', 19),
 ('EF', 9),
 ('antiguedad', 105),
 ('Mes', 1313),
 ('Hl_sum_prodSim_3M', 57),
 ('Dcto_porc_prodSim_3M', 71),
 ('Hl_sum_prodSim_6M', 32),
 ('Dcto_porc_prodSim_6M', 23),
 ('Hl_sum_prodSim_1M', 41),
 ('Dcto_porc_prodSim_1M', 36),
 ('avg_6M_exec', 746)]